# Tinyflux: Example with fictious bank transactions

In [2]:
import csv
from tinyflux import TinyFlux, Point, FieldQuery, TagQuery, TimeQuery, MeasurementQuery
from datetime import datetime, timezone, timedelta

import re

## Initialize the TinyFlux database

In [3]:
db = TinyFlux("swiss_bank_transactions.db")

## Writing a CSV file into Tinyflux

In [25]:
# Read CSV and insert data into TinyFlux
with open("example_data/swiss_bank_transactions.csv", mode="r") as file:
    reader = csv.DictReader(file)
    for row in reader:
        point = Point(
            time=datetime.strptime(row["time"], "%Y-%m-%d %H:%M:%S"),
            measurement="transactions",
            fields={"amount": float(row["amount"])},  # Only numeric fields
            tags={
                "location": row["location"],
                "account_id": row["account_id"],
                "currency": row["currency"],  # Move non-numeric fields to tags
                "transaction_type": row["transaction_type"]
            }
        )
        db.insert(point)

## Writing manually into Tinyflux

In [23]:
db.remove_all()

In [5]:
# Create manual records
p1 = Point(
    time=datetime(2024, 1, 1, 12, 0, tzinfo=timezone.utc),
    tags={"location": "Winterthur", "account_id": "1234", "currency": "CHF", "transaction_type": "deposit"},
    fields={"amount": 1000.0}
)

p2 = Point(
    time=datetime(2024, 1, 2, 12, 0, tzinfo=timezone.utc),
    tags={"location": "Winterthur", "account_id": "1234", "currency": "CHF", "transaction_type": "deposit"},
    fields={"amount": 2000.0}
)

# Insert into the DB.
db.insert_multiple([p1, p2])

2

## Querying by Time

In [6]:
# Search for a time value
Time = TimeQuery()
time_start = Time >= datetime(2023, 1, 1, tzinfo=timezone.utc)
time_end = Time < datetime(2023, 12, 31, tzinfo=timezone.utc)
time_end_2 = db.get(time_end)
time_count = db.count(time_start & time_end)
print("\nTime Query Count:")
print(time_count)

# search for values with filter on time
timebsp = db.search(Time > datetime(2023, 9, 1, tzinfo=timezone.utc))

for result in timebsp:
    print(result)


# Querying by Measurment

Measurement = MeasurementQuery()
measur = db.search(Measurement == "transactions")

print(measur)






Time Query Count:
6000
Point(time=2024-01-01T12:00:00+00:00, measurement=_default, tags=location:Winterthur; account_id:1234; currency:CHF; transaction_type:deposit, fields=amount:1000.0)
Point(time=2024-01-01T12:00:00+00:00, measurement=_default, tags=location:Winterthur; account_id:1234; currency:CHF; transaction_type:deposit, fields=amount:1000.0)
Point(time=2024-01-01T12:00:00+00:00, measurement=_default, tags=location:Winterthur; account_id:1234; currency:CHF; transaction_type:deposit, fields=amount:1000.0)
Point(time=2024-01-01T12:00:00+00:00, measurement=_default, tags=location:Winterthur; account_id:1234; currency:CHF; transaction_type:deposit, fields=amount:1000.0)
Point(time=2024-01-01T12:00:00+00:00, measurement=_default, tags=location:Winterthur; account_id:1234; currency:CHF; transaction_type:deposit, fields=amount:1000.0)
Point(time=2024-01-02T12:00:00+00:00, measurement=_default, tags=location:Winterthur; account_id:1234; currency:CHF; transaction_type:deposit, fields=a

## Querying by Field

In [27]:
from collections import defaultdict
import pandas as pd

# Search for a field value
Field = FieldQuery()
#Tag = TagQuery()
#field_results = db.select("fields.amount", Field.amount > 1000.0)
field_results = db.search(Field.amount < 1000.0)
#field_select = field_results.select("tag.location","tag.account")
#selected_results = field_results.select("tag.location", "tag.account", "fields.amount")


#print("\nField Query Results:")
#for result in field_results:
#    print(result)


# and operator
field_results2 = db.search((Field.amount < 5000.0) & (Field.amount > 0.0))
#print("\nField Query Results2:")
#for result in field_results2:
#    print(result)




# Initialize a dictionary to store the grouped sums
amount_sums = defaultdict(float)
transaction_counts = defaultdict(int)

# Iterate through the filtered results (Point objects)
for result in field_results2:
    # Access the 'transaction_type' tag directly from the 'tags' dictionary
    transaction_type = result.tags['transaction_type']

    # Access the 'amount' field directly from the 'fields' dictionary
    amount = float(result.fields['amount'])  # Ensure it's treated as a float

    # Debug: Print the transaction type and amount for verification
    print(f"Transaction Type: {transaction_type}, Amount: {amount}")

    # Add the amount to the corresponding transaction_type group
    amount_sums[transaction_type] += amount

    # Increment the count of transactions for that transaction_type
    transaction_counts[transaction_type] += 1

# Calculate and print the average for each transaction_type
for transaction_type in amount_sums:
    total_sum = amount_sums[transaction_type]
    count = transaction_counts[transaction_type]
    average_amount = total_sum / count
    print(f"Transaction Type: {transaction_type}, Total Sum: {total_sum}, Count: {count}, Average Amount: {average_amount}")


'''
# Create a list of dictionaries to hold the data
data = []

# Populate the list with data from field_results2
for result in field_results2:
    transaction_type = result.tags['transaction_type']
    amount = float(result.fields['amount'])
    data.append({'transaction_type': transaction_type, 'amount': amount})

# Create a DataFrame from the list
df = pd.DataFrame(data)

# Group by 'transaction_type' and calculate the average amount
averages = df.groupby('transaction_type', as_index=False).agg(
    total_amount=('amount', 'sum'),  # Optional: Total amount
    average_amount=('amount', 'mean'),  # Average amount
    count=('amount', 'count')  # Count of transactions
)

# Print the results
print(averages)
'''





Transaction Type: withdrawal, Amount: 3415.86
Transaction Type: withdrawal, Amount: 3669.64
Transaction Type: transfer, Amount: 1107.44
Transaction Type: deposit, Amount: 2784.23
Transaction Type: transfer, Amount: 3367.41
Transaction Type: deposit, Amount: 1084.57
Transaction Type: deposit, Amount: 2457.76
Transaction Type: deposit, Amount: 219.92
Transaction Type: deposit, Amount: 196.43
Transaction Type: transfer, Amount: 2738.98
Transaction Type: transfer, Amount: 2039.86
Transaction Type: deposit, Amount: 2291.68
Transaction Type: withdrawal, Amount: 1496.46
Transaction Type: transfer, Amount: 1794.01
Transaction Type: transfer, Amount: 2540.7
Transaction Type: withdrawal, Amount: 1467.73
Transaction Type: deposit, Amount: 4925.25
Transaction Type: withdrawal, Amount: 1313.0
Transaction Type: withdrawal, Amount: 150.51
Transaction Type: deposit, Amount: 3356.88
Transaction Type: withdrawal, Amount: 3313.69
Transaction Type: withdrawal, Amount: 4006.41
Transaction Type: withdrawal,

"\n# Create a list of dictionaries to hold the data\ndata = []\n\n# Populate the list with data from field_results2\nfor result in field_results2:\n    transaction_type = result.tags['transaction_type']\n    amount = float(result.fields['amount'])\n    data.append({'transaction_type': transaction_type, 'amount': amount})\n\n# Create a DataFrame from the list\ndf = pd.DataFrame(data)\n\n# Group by 'transaction_type' and calculate the average amount\naverages = df.groupby('transaction_type', as_index=False).agg(\n    total_amount=('amount', 'sum'),  # Optional: Total amount\n    average_amount=('amount', 'mean'),  # Average amount\n    count=('amount', 'count')  # Count of transactions\n)\n\n# Print the results\nprint(averages)\n"

## Querying by Tag

In [29]:
# Search for a tag value
Tag = TagQuery()
tag_results = db.search(Tag.location == 'Zurich')
print("Tag Query Results:")
#for result in tag_results:
#    print(result)


# mit Match values
tagresutls = db.search(Tag.location.matches('.*Zur.*'))
'''
for results in tagresutls:
    print(results)
'''
    
# Initialize a dictionary to store the grouped sums
grouped_data = defaultdict(float)

# Iterate through the filtered results (Point objects)
for result in tag_results:
    # Access the 'transaction_type' tag directly from the 'tags' dictionary
    transaction_type = result.tags['transaction_type']

    # Access the 'amount' field directly from the 'fields' dictionary
    amount = result.fields['amount']

    # Add the amount to the corresponding transaction_type group
    grouped_data[transaction_type] += float(amount)

# Print the grouped sums
for transaction_type, total_sum in grouped_data.items():
    print(f"Transaction Type: {transaction_type}, Total Amount: {total_sum}")



Tag Query Results:
Transaction Type: deposit, Total Amount: 197057.75
Transaction Type: transfer, Total Amount: 162100.46000000002
Transaction Type: withdrawal, Total Amount: 208518.9299999999
